# Исследование базы данных StackOverflow

В данном проекте будет работа с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.

## Часть 1

1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(p.id)
FROM stackoverflow.posts AS p
LEFT JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
WHERE pt.type = 'Question' AND (score > 300 OR favorites_count >= 100)

2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(questions_cnt), 0)
FROM
    (SELECT p.creation_date::date,
           COUNT(p.id) AS questions_cnt
    FROM stackoverflow.posts AS p
    LEFT JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
    WHERE pt.type = 'Question' AND p.creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'
    GROUP BY 1) AS q 

3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT(u.id))
FROM stackoverflow.users AS u 
LEFT JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE u.creation_date::date = b.creation_date::date

4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
WITH votes_count AS (
    SELECT p.id AS id_post,
           COUNT(v.id) AS votes_cnt
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON p.user_id = u.id
    JOIN stackoverflow.votes AS v ON v.post_id = p.id
    WHERE display_name = 'Joel Coehoorn'
    GROUP BY p.id 
      )
SELECT COUNT(id_post)
  FROM votes_count
  WHERE votes_cnt >=1

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types 
ORDER BY id

6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT u.id AS user_id,
       COUNT(v.id) AS votes_cnt
FROM stackoverflow.users AS u
JOIN stackoverflow.votes AS v ON u.id = v.user_id
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY votes_cnt DESC, user_id DESC
LIMIT 10

7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(id) AS b_cnt,
       DENSE_RANK() OVER (ORDER BY COUNT(id) DESC)
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY b_cnt DESC, user_id
LIMIT 10

8. Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title != '' AND score != 0

9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE user_id IN
    (SELECT user_id
    FROM stackoverflow.badges
    GROUP BY user_id
    HAVING COUNT(id) > 1000)
AND title IS NOT NULL

10. Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           WHEN views < 100 THEN 3
           END AS group_number
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views != 0
ORDER BY views DESC

11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH users_us AS
    (
    SELECT id,
           views,
           CASE
               WHEN views >= 350 THEN 1
               WHEN views < 350 AND views >= 100 THEN 2
               WHEN views < 100 THEN 3
               END AS group_number
    FROM stackoverflow.users
    WHERE location LIKE '%United States%' AND views != 0
    ORDER BY views DESC
    )

SELECT id,
       group_number,
       views
FROM       
    (SELECT *,
           MAX(views) OVER (PARTITION BY group_number ORDER BY views DESC) AS max_views
    FROM users_us) AS top_us
WHERE views = max_views
ORDER BY views DESC, id

12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH users_cnt_nov AS (
    SELECT EXTRACT(DAY FROM creation_date) AS day,
           COUNT(id) AS users_cnt
    FROM stackoverflow.users
    WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
    GROUP BY EXTRACT(DAY FROM creation_date)
    )
SELECT day,
       users_cnt,
       SUM(users_cnt) OVER (ORDER BY day)
FROM users_cnt_nov 

13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT(p.user_id),
       MIN(p.creation_date) OVER (PARTITION BY p.user_id) - u.creation_date AS interval
FROM stackoverflow.posts AS p
LEFT JOIN stackoverflow.users AS u ON p.user_id = u.id

## Часть 2

1. Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date,
       SUM(views_count)
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY 1
ORDER BY 2 DESC

2. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id = p.user_id
JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
WHERE p.creation_date::date BETWEEN u.creation_date::date AND u.creation_date::date + INTERVAL '1 month'
AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(DISTINCT p.id) > 100

3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date AS post_month,
       COUNT(DISTINCT id) post_cnt
FROM stackoverflow.posts
WHERE user_id IN (
    SELECT DISTINCT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON u.id = p.user_id
    WHERE u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30'
    AND p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31'
    )
AND EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY 1
ORDER BY 1 DESC

4. Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY 1, 2

5. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
SELECT ROUND(AVG(days_cnt))
FROM 
    (SELECT user_id,
           COUNT(DISTINCT creation_date::date) days_cnt
    FROM stackoverflow.posts
    WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
    GROUP BY user_id) AS active_days

6. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

In [ ]:
SELECT posts_month,
       posts_cnt,
       ROUND( (posts_cnt - LAG(posts_cnt) OVER () )* 100/ LAG(posts_cnt) OVER () :: numeric, 2)
FROM       
    (SELECT EXTRACT(MONTH FROM creation_date::date) AS posts_month,
           COUNT(id) AS posts_cnt
    FROM stackoverflow.posts
    WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
    GROUP BY EXTRACT(MONTH FROM creation_date::date)
     ) AS raw

7. Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH week AS (   
      SELECT EXTRACT(WEEK FROM creation_date) AS week_number,
             MAX(creation_date) OVER (ORDER BY EXTRACT(WEEK FROM creation_date))
        FROM stackoverflow.posts 
       WHERE user_id = (
                 SELECT user_id
                   FROM stackoverflow.posts 
               GROUP BY user_id 
               ORDER BY COUNT(id) DESC
                  LIMIT 1
                     ) 
        AND creation_date::date BETWEEN '2008-10-01' AND '2008-10-31'          
   ORDER BY creation_date
             )
   
SELECT DISTINCT *
FROM week
ORDER BY week_number